<div style="
    border: 2px solid #4CAF50; 
    padding: 15px; 
    background-color: #f4f4f4; 
    border-radius: 10px; 
    align-items: center;">

<h1 style="margin: 0; color: #4CAF50;">Neural Networks: Datasets und Dataloader</h1>
<h2 style="margin: 5px 0; color: #555;">DSAI</h2>
<h3 style="margin: 5px 0; color: #555;">Jakob Eggl</h3>

<div style="flex-shrink: 0;">
    <img src="https://www.htl-grieskirchen.at/wp/wp-content/uploads/2022/11/logo_bildschirm-1024x503.png" alt="Logo" style="width: 250px; height: auto;"/>
</div>
<p1> © 2025/26 Jakob Eggl. Nutzung oder Verbreitung nur mit ausdrücklicher Genehmigung des Autors.</p1>
</div>
<div style="flex: 1;">
</div>   

# Datasets und Dataloader (in PyTorch)

In diesem Notebook wollen wir uns den sogenannten Dataloadern und den dazugehörigen Datasets widmen. Wir werden dabei zuerst die Notwendigkeit solcher Objekte besprechen und uns im Anschluss deren Erstellung betrachten. Am Ende werden wir noch Vorteile und Nachteile besprechen.

### Warum benötigen wir diese Objekte?

Der Grund, warum wir `torch.utils.data.Dataset` und `torch.utils.data.DataLoader` verwenden werden, bezieht sich auf folgende Punkte.

1. Zu großes Dataset:
    * Falls unser Dataset zu groß für die CPU (also für den RAM) bzw. für die GPU (also für den VRAM) ist, dann müssen wir das Dataset aufteilen in kleinere Teile
    * Es kann natürlich auch händisch geteilt werden, wir werden aber eine elegantere Lösung (mit *Datasets* und/oder *DataLoader*) kennen lernen.

2. Vorverarbeitungsschritte sind kompliziert bzw. nur temporär:
    * Falls unsere Daten in ein bestimmtes Format gebracht werden müssen, dann wollen wir das oft nicht permament machen. Beispiele: Data-Augmentation, Downscaling bei Bildern oder Text in Embedding-Vektoren umwandeln.
    * Wir wollen also nicht unser Änderungen auch abspeichern bzw. sogar unsere Daten mit den Änderungen ersetzen.

3. Art der Daten lassen das "gesamte Laden" nicht zu:
    * Für Bilder/Videos/Text ist es schwierig, diese überhaupt sinnvoll gleichzeitig zu öffnen. Natürlich ist es möglich, diese als einen großen Tensor/Array zu speichern, jedoch wird so die Vorverarbeitung und weitere Verwendung auch etwas kompliziert.

Alle diese Punkte schließen nicht aus, dass wir einfach wie bisher einen Teil unserer Daten laden, diese vorverarbeiten und im Anschluss an unser Modell übergeben, bis wir wieder die nächsten Daten laden usw.

Jedoch ist dies einerseits umständlich und andererseits langsam. Das Training eines neuronalen Netzes nimmt nämlich einige Zeit in Anspruch und wir wollen so wenig Zeit wie möglich unsere GPU's "unbeschäftigt" lassen. Wir werden sehen, dass wir sowohl Speicher als auch Runtime mit diesen Methoden minimieren können.

**Kurz gesagt:** Die Kombination aus **Dataset** und **Dataloader** erlaubt es uns, die Datenbereitstellung sauber von der Model-Klasse bzw. dem späteren Trainingsvorgang zu separieren und somit Speicher- und Zeiteffizient zu arbeiten.

## Datasets

[PyTorch_Dataset_Documentation](https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.Dataset)

Ein Dataset ist eine Python Klasse, die von `torch.utils.data.Dataset` erbt und 3 Methoden implementiert. Wir starten mit einem Beispiel:

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.data = torch.from_numpy(data).float()
        self.targets = torch.from_numpy(targets).long()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

Wir können nun unser Dataset initialisieren und die Funktionsweise testen.

In [3]:
data = np.random.randn(100, 5)
targets = np.random.randint(0, 3, size=(100,))

In [4]:
dataset = MyDataset(data, targets)
loader = DataLoader(dataset, batch_size=10, shuffle=True)

In [5]:
for x, y in loader:
    print(x)
    print(y)

tensor([[ 0.5069,  0.1092, -0.2384, -0.2577,  0.8052],
        [-0.3121,  0.2231,  0.5598, -1.4180,  0.1089],
        [ 1.2742, -1.1464, -0.4507, -0.4376, -2.4784],
        [ 0.2624, -1.1176,  0.5151,  2.4745, -1.0527],
        [ 2.0679, -0.5308, -0.6077,  1.0965, -1.0772],
        [-0.2201,  0.7058, -0.1247,  0.7160, -0.3400],
        [-2.1532,  1.6777,  1.1952, -0.4793,  1.0680],
        [ 0.3298, -0.8381, -0.3389, -0.1819,  0.7724],
        [-0.7027,  0.2146, -0.8227, -0.0369,  0.3882],
        [ 0.1065,  0.4170, -1.2056,  0.1908,  1.0696]])
tensor([0, 2, 2, 1, 0, 1, 0, 2, 0, 2])
tensor([[ 0.9685, -2.3074,  0.3342, -0.0494, -1.1488],
        [-0.5176, -0.1098,  1.6226,  1.5894,  0.8609],
        [-0.0244,  0.1765, -0.3933, -0.1908, -0.2194],
        [ 0.0610,  1.4928,  1.0841, -0.2602,  0.8757],
        [ 0.3410, -0.3119,  1.4237,  0.7171, -0.8697],
        [-0.1329,  0.0656, -1.3961,  1.7001,  0.5371],
        [-0.9989, -0.5213, -0.7923, -0.4502,  0.6730],
        [ 0.0975, -2.0438

Hier haben wir schon etwas vorgegriffen und bereits einen `DataLoader` genutzt, dieser wird später noch genauer erklärt.

Von uns gibt es also folgende Methoden zu implementieren:
* `__init__(self, data, targets)` (Konstruktor): Bekommt die Daten übergeben (in welcher Form auch immer)
* `__len__(self)`: Muss die Anzahl der Daten in dem Dataset liefern. Meist ein einfacher `return len(self.data)` Befehl.
* `__getitem__(self, idx)`: Für einen gegebenen Index `idx` soll *ein* Datenpunkt returned werden. Dieser kann aber auch ein Tupel sein (zum Beispiel werden wir immer gleich $X$ (Daten) und Label $y$ übergeben).

Hier könnte man dann statt dem obigen Testbeispiel von uns ein Pandas Dataframe übergeben.

Sehen wir uns nun noch weitere Beispiele an.

**Anderes Beispiel:**

Was, wenn wir ein Bilder Dataset haben? Sprich einen wir haben einen Ordner, in welchem unsere Daten in der Form von Bildern abgespeichert sind.

In [6]:
from PIL import Image
from torchvision import transforms
import os
import glob
import matplotlib.pyplot as plt

In [7]:
class MyImageFolderDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform or transforms.ToTensor()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        label = torch.tensor(self.labels[idx])
        if self.transform:
            img = self.transform(img)
        return img, label

Hier übergeben wir in der `__init__` Methode 2 Argumente für die Daten und noch ein zusätzliches Argument für etwaige Transformationen. Wir sehen also, dass wir hier im Dataset auch die Daten dementsprechend *transformieren* können. Dazu zählt zum Beispiel das Downscalen (zum Beispiel auf die Größe $100\times 100$, das Transformieren in Schwarz-Weiß Bilder oder das Data-Augmentieren.)

Wir können das jetzt selber testen. Übergebt dafür einfach einen Ordner mit dem entsprechenden Pfad. 

In [8]:
image_folder_path = os.path.join("..", "private", "katzen", "*.jpg")
label_path = os.path.join("..", "private", "katzen", "labels.txt")

image_paths = glob.glob(image_folder_path)

with open(label_path, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

FileNotFoundError: [Errno 2] No such file or directory: '../private/katzen/labels.txt'

In [ ]:
image_paths

In [ ]:
len(image_paths)

In [ ]:
labels

Die Labels müssten noch umgewandelt werden in Zahlen (oder One-Hot-Vektoren), falls wir noch weiter arbeiten wollen damit.

In [ ]:
# Generate unique ints for each label
unique_labels = list(set(labels))
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
labels = [label_to_int[label] for label in labels]

In [ ]:
labels

Wir erstellen uns nun ein PyTorch Dataset aus diesen Daten.

In [ ]:
my_cat_dataset = MyImageFolderDataset(image_paths, labels)

In [ ]:
cat_loader = DataLoader(my_cat_dataset, batch_size=1, shuffle=False)

In [ ]:
for imgs, lbls in cat_loader:
    fig, axes = plt.subplots(figsize=(8, 4)) # only one plot
    axes.imshow(np.transpose(imgs[0].numpy(), (1, 2, 0)))
    axes.set_title(f"Label: {lbls[0].item()}")
    axes.axis('off')
    plt.show()

Warum können wir hier nicht 2 Bilder (*batch_size*) gleichzeitig verwenden? --> **Probieren wir es einfach aus im Code.**

## Transformationen in PyTorch Datasets

Mit Hilfe von uns definierten, sogenannten **transformations** können wir unsere Daten sehr elegant transformieren. Eine beispielshafte Transformations-Pipeline sieht folgendermaßen aus.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((100, 100)),
    transforms.GaussianBlur(3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
])

**Hinweis:** Die einzelnen Transformationen werden der Reihe nach abgearbeitet, also zuerst zu einem Tensor verwandelt, danach die Größe auf $100 \times 100$ angepasst usw.

In [ ]:
my_transformed_cat_dataset = MyImageFolderDataset(image_paths, labels, transform=transform)

In [ ]:
batch_size = 2
transformed_cat_loader = DataLoader(my_transformed_cat_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# plot all images in a batch for all batches next to each other. Each batch in a new row.
for imgs, lbls in transformed_cat_loader:
    fig, axes = plt.subplots(1, batch_size, figsize=(8, 4)) # only one plot
    for i in range(batch_size):
        axes[i].imshow(np.transpose(imgs[i].numpy(), (1, 2, 0)))
        axes[i].set_title(f"Label: {lbls[i].item()}")
        axes[i].axis('off')
    plt.show()

**Hinweis:** Bei jeder Ausführung vom Dataloader (wenn er bereits alle Daten ausgegeben hat), liefert neue Daten. Somit können wir die Data Augmentation Techniken gut verwenden, insbesondere wenn wir mehrere Epochen (=Durchläufe des ganzen Datasets) haben. 

Natürlich wird unser Modell die Bilder nie direkt zu sehen bekommen. Wir plotten sie hier, um den Effekt zu sehen.

Das Modell wird die Daten in folgender Form bekommen.

In [ ]:
for image, label in transformed_cat_loader:
    print(image.shape)
    print(label.shape)

    print(image)
    print(label)

**Hinweis:** Es werden die Transformationen mittels `transformation` library meistens nur für Bilder angewendet. Für tabellarische Daten können wir die Befehle mit `numpy` (und/oder `pandas`) selber implementieren.

Wir betrachten jetzt noch ein weiteres Beispiel, welches uns zeigt, dass wir auch vorgefertigte Datasets verwenden können.

### Vorgefertigte Datasets

In [ ]:
from torchvision import datasets
import torchvision

In [ ]:
path = os.path.join("..", "..", "_data", "private", "cifar10")

cifar_10_dataset = datasets.CIFAR10(root=path, train=True, download=True, transform=transforms.ToTensor()) # other possibility would be train=False, i.e. the test dataset

Wir laden uns also hier das **CIFAR10** Dataset herunter. Wir sehen auch, dass wir hier eine Flag `train=True` haben, somit gibt es auch ein eigenes Test-Dataset. Das ist auch später unsere Vorgehensweise.

Die Daten werden hier im Angegeben Pfad gespeichert (und müssen natürlich beim nächsten Mal ausführen nicht wieder heruntergeladen werden).

![Cifar10](../resources/Cifar10.jpg)

(from https://www.cs.toronto.edu/~kriz/cifar.html)

### Tensordatasets

Eine weitere Möglichkeit, ein PyTorch-Dataset zu ersetllen, ohne eine eigene Klasse machen zu müssen, ist das sogenannte `Tensordataset`.

Das Tensordataset erlaubt es, wenn unsere Daten (Input und Label) schon in Matrix- bzw. Vektorform (quasi Tabellen) sind.

**Wie sieht das in PyTorch aus?**

In [ ]:
from torch.utils.data import TensorDataset
import torch

Nehmen wir an, wir haben folgende Daten:

In [ ]:
X = torch.randn(2000, 5)
y = torch.randint(0, 2, (2000,)) # 0 or 1

In [ ]:
X

In [ ]:
y

Dann können wir nun folgendermaßen ein **Tensor**Dataset erstellen:

In [ ]:
dataset = TensorDataset(X, y)

Dieses können wir nun auch wieder in einen Dataloader geben usw., wird hier jedoch nicht mehr genauer demonstriert.

### Vorteile und Nachteile:

**Vorteile:**
* Bisherige Dataframes etc. können recht einfach in ein PyTorch Dataset gebracht werden (vorausgesetzt es sind nur numerische Datentypen)
* Erlaubt auch für mehrere Tensoren, zum Beispiel (Input, Label, Maske).

**Nachteile:**
* Alles ist im RAM
* Daten müssen vorher schon verarbeitet werden und können nicht "on the fly" noch angepasst (normalisiert, augmented usw.) werden

---

Nun kommen wir zu den sogenannten Dataloaders in Python (bzw. PyTorch).

## DataLoader (in PyTorch)

Nachdem wir ein Dataset erstellt haben (oder heruntergeladen), können wir diesem einem Dataloader übergeben.

Der Dataloader in PyTorch hat folgende wichtige Argumente:
* `dataset`: Das Dataset, welches "Batchweise" zurück gegeben werden soll
* `batch_size`: Anzahl der Elemente aus dem Dataset, die pro Aufruf zurück gegeben werden sollen
* `shuffle`: Flag, welche angibt, ob die Daten durchgemischt ("geshuffled") werden sollen
* `num_workers`: (Optional) Wie viele Subprozesse im Hintergrund arbeiten sollen, um die Daten vorzubereiten (**Achtung:** Kann bei Windows durchaus zu großen Problemen (zum Beispiel Freezen der Trainingsmethode) führen)
* `collate_fn`: (Optional) Gibt an, wie unsere Daten zu einem Stack kombiniert werden sollen.
* $\vdots$

**Hinweis:** Es sind eigentlich noch mehr Argumente *optional*, jedoch ist es gut, wenn wir die nicht-optionalen Argumente explizit angeben.

**Hinweis:** Die `collate_fn` Funktion werden wir bei unserem Projekt bzgl. *Image Inpainting* benötigen.

**Wichtig:** Es gibt für **jedes (Teil)dataset**, also für Train- und für Testset sowohl ein **eigenes Dataset**, als auch einen **eigenen Dataloader**!

Nachdem der Dataloader bereits weiter oben verwendet worden ist, wollen wir hier nochmal kurz seine Verwendung in einem kleinem Code-Ausschnitt zeigen.

In [ ]:
dataloader_cifar10 = DataLoader(cifar_10_dataset, batch_size=4, shuffle=True)

In [ ]:
for images, labels in dataloader_cifar10:
    # Make a grid of the images
    grid = torchvision.utils.make_grid(images)
    
    # Convert tensor to numpy and plot
    plt.imshow(grid.permute(1, 2, 0))  # (C, H, W) -> (H, W, C)
    plt.axis('off')
    plt.show()
       
    break

### Vorteile und Nachteile von Datasets und DataLoader

**Vorteile:**
* Sauberer, einfacher Weg, Daten gut zu organisieren

**Nachteile:**
* Um nun ein Modell und deren Trainingsspezifikationen (welches Dataset, welche Hyperparameter usw.) muss nun auch angegeben werden, wie genau die Dataloader ausgesehen haben.

Natürlich erspart uns die Dataset/Dataloader Kombination leider nicht, dass wir uns um die Qualität der Daten kümmern müssen.

Daten können nach wie vor:
* Fehlen
* Falsch sein
* Zu wenig sein
* Unnormalisiert sein
* usw.

![Reel_NaN_Values](../resources/Instagram_Reel_NaNs.mp4)

(von https://www.instagram.com/reel/DMujUkYBXv3/?igsh=eG1uMDl0MHdvM3B2)